In [82]:
import sys
sys.path.append('..')
from api.api import PDTB
from constituent import Constituent
import itertools

In [14]:
pdtb2 = '/home/pengfei/data/pdtb_v2/all/conll/'
pdtb3 = PDTB()

## only for one individual sentence for explicit

In [167]:
def get_constituents(docid, sentid, conn_index):
    """
    Args:
            docid(str)
            sentid(int)
            conn_index(list[int])
    Returns:
            [ [(token_index, text),...], ...]"""
    syntax_tree = pdtb3.get_syntax_tree(docid, sentid)
    if syntax_tree.tree == None:
        return []
    all_leaves = syntax_tree.tree.get_leaves()
    conn_indices = conn_index
    constituent_nodes = []
    if len(conn_indices) == 1:# like and or so...
        conn_node = syntax_tree.get_leaf_node_by_token_index(conn_indices[0]).up
    else:
        conn_node = syntax_tree.get_common_ancestor_by_token_indices(conn_indices)
        conn_leaves = set([all_leaves.index(syntax_tree.get_leaf_node_by_token_index(conn_index)) for conn_index in conn_indices])
        children = conn_node.get_children()
        for child in children:
            leaves = set([all_leaves.index(n) for n in child.get_leaves()])
            if list(leaves-conn_leaves)!=[]: constituent_nodes.append(list(leaves-conn_leaves))
    curr = conn_node
    while not curr.is_root():
        sibs = [ [all_leaves.index(n) for n in sib] for sib in syntax_tree.get_siblings(curr)]
        constituent_nodes.extend(sibs)
        curr = curr.up

    tokens_indices_with_text = pdtb3.get_tokens_indices_with_text(docid, sentid)
    subtree_list = [ [(o,tokens_indices_with_text[o][2]) for o in node] for node in constituent_nodes]
    return subtree_list

def merge_consti(subtree_list):
    return {(o[0][0],o[-1][0]):i for i,o in enumerate(subtree_list)}

def arg_is_contained(i, arg):
    docid = pdtb3.parse_data[i]['DocID']
    sentid = pdtb3.get_arg_sent_id(i, arg)
    conn_indices = [o[1] for o in pdtb3.get_arg_token_list(i, 'Connective')]
    subtree_list = get_constituents(docid, sentid[0], conn_indices)
    merged_consti_list = merge_consti(subtree_list)
    token_list = pdtb3.get_arg_token_list(i, arg)
    start,end = token_list[0][1], token_list[-1][1]
    # filter results
    consti_list = [k for k in merged_consti_list.keys() if k[0]>=start and k[1]<=end]
    # generate results
    results = []
    for i in range(1, len(consti_list)+1):
        for subset in itertools.combinations(consti_list, i):
            results.append(expand(subset))
    # check
    return [o[1] for o in token_list] in results

def expand(subset):
    ret = []
    for subsubset in subset:
        for i in range(subsubset[0], subsubset[1]+1):
            ret.append(i)
    return sorted(ret)

In [159]:
arg_is_contained(34, 'Arg1')
# pdtb3.get_arg_token_list(6, 'Arg1')

False

In [172]:
bad = 0
total =0
for i in range(1000):
    if pdtb3.parse_data[i]['Type'] == 'Explicit' and pdtb3.get_arg_sent_id(i,'Arg1')[0] == pdtb3.get_arg_sent_id(i,'Arg2')[0]:
        total+=1
        if not arg_is_contained(i, 'Arg1'):
            bad += 1
            print(pdtb3.get_highlighted_parsetree(i, v=True))
bad/total

Relation: 34,  Green for Connective(if any), Red for Arg1 and Blue for Arg2. 
DocID: wsj_0739,	Type: Explicit,	Sense: Comparison.Concession.Arg2-as-denier,
Arg1_sent_id: 13	,	Arg2_sent_id: 13

         /- /-The
      /-|
     |   \- /-researchers
   /-|
  |  |   /- /-said
  |   \-|
  |     |      /- /- /-they
  |      \- /-|
  |           |   /- /-did
  |            \-|
  |               \- /- /-this
  |
  |-- /-*COMMA*
  |
  |-- /-but
  |
  |      /- /-the
  |   /-|
--|  |   \- /-toxin
  |  |
  |--|   /- /-did
  |  |  |
  |  |  |-- /-n't
  |   \-|
  |     |   /- /-induce
  |     |  |
  |      \-|-- /- /-immunity
  |        |
  |        |   /- /-to
  |         \-|
  |           |      /- /-whooping
  |            \- /-|
  |                  \- /- /-cough
  |
   \- /-.

Relation: 46,  Green for Connective(if any), Red for Arg1 and Blue for Arg2. 
DocID: wsj_0777,	Type: Explicit,	Sense: Expansion.Conjunction,
Arg1_sent_id: 0	,	Arg2_sent_id: 0

      /- /-SYDNEY-Qintex
     |
   /-|-- /-A

0.11437908496732026

In [171]:
print(pdtb3.get_highlighted_parsetree(975, v=True))

Relation: 975,  Green for Connective(if any), Red for Arg1 and Blue for Arg2. 
DocID: wsj_0730,	Type: Explicit,	Sense: Contingency.Cause.Reason,
Arg1_sent_id: 37	,	Arg2_sent_id: 37
None



In [32]:
pdtb3.parse_data[106]

{'DocID': 'wsj_0799',
 'ID': 106,
 'Type': 'Explicit',
 'Sense': ['Expansion.Conjunction'],
 'Connective': {'CharacterSpanList': [[5809, 5817], [5878, 5886]],
  'RawText': 'not only but also',
  'TokenList': [[5809, 5812, 1089, 46, 2],
   [5813, 5817, 1090, 46, 3],
   [5878, 5881, 1097, 46, 10],
   [5882, 5886, 1098, 46, 11]]},
 'Arg1': {'CharacterSpanList': [[5818, 5876]],
  'RawText': 'merged three military-electronics manufacturing operations',
  'TokenList': [[5818, 5824, 1091, 46, 4],
   [5825, 5830, 1092, 46, 5],
   [5831, 5851, 1093, 46, 6],
   [5852, 5865, 1094, 46, 7],
   [5866, 5876, 1095, 46, 8]]},
 'Arg2': {'CharacterSpanList': [[5887, 5976]],
  'RawText': 'closed an unrelated plant that makes ordnance devices used in fighter planes and missiles',
  'TokenList': [[5887, 5893, 1099, 46, 12],
   [5894, 5896, 1100, 46, 13],
   [5897, 5906, 1101, 46, 14],
   [5907, 5912, 1102, 46, 15],
   [5913, 5917, 1103, 46, 16],
   [5918, 5923, 1104, 46, 17],
   [5924, 5932, 1105, 46, 18],


In [8]:
# only for one individual sentence for implicit
# for PS case for explicit
# for PS case for implicit

In [30]:
for i, r in enumerate(pdtb3.parse_data):
    if r['Type'] != 'Explicit':
        continue
    if len(r['Connective']['TokenList']) > 2:
        print(r)
        print(i)
        break

{'DocID': 'wsj_0799', 'ID': 106, 'Type': 'Explicit', 'Sense': ['Expansion.Conjunction'], 'Connective': {'CharacterSpanList': [[5809, 5817], [5878, 5886]], 'RawText': 'not only but also', 'TokenList': [[5809, 5812, 1089, 46, 2], [5813, 5817, 1090, 46, 3], [5878, 5881, 1097, 46, 10], [5882, 5886, 1098, 46, 11]]}, 'Arg1': {'CharacterSpanList': [[5818, 5876]], 'RawText': 'merged three military-electronics manufacturing operations', 'TokenList': [[5818, 5824, 1091, 46, 4], [5825, 5830, 1092, 46, 5], [5831, 5851, 1093, 46, 6], [5852, 5865, 1094, 46, 7], [5866, 5876, 1095, 46, 8]]}, 'Arg2': {'CharacterSpanList': [[5887, 5976]], 'RawText': 'closed an unrelated plant that makes ordnance devices used in fighter planes and missiles', 'TokenList': [[5887, 5893, 1099, 46, 12], [5894, 5896, 1100, 46, 13], [5897, 5906, 1101, 46, 14], [5907, 5912, 1102, 46, 15], [5913, 5917, 1103, 46, 16], [5918, 5923, 1104, 46, 17], [5924, 5932, 1105, 46, 18], [5933, 5940, 1106, 46, 19], [5941, 5945, 1107, 46, 20], [

In [85]:
import itertools

stuff = [1, 2, 3]
for L in range(0, len(stuff)+1):
    for subset in itertools.combinations(stuff, 1):
        print(subset)

(1,)
(2,)
(3,)
(1,)
(2,)
(3,)
(1,)
(2,)
(3,)
(1,)
(2,)
(3,)
